# Animations

Generate animation of centerline across all time

* Worm centerline
    * all centerline across time
    * animation of centerlines
    * animation of centerlines with neural information

In [ ]:
import numpy as np
from scipy.interpolate import interp1d
from scipy.interpolate import interpn
from matplotlib import animation, pyplot, gridspec as plt
import seaborn as sns
import scipy.io as sio

%pylab inline

## Display all centerline across time
On a single plot.

In [ ]:
# Read in worm data from SharedData .npz file
Worm = 5
worm_data = np.load('../SharedData/Worm'+str(Worm)+'.npz')

rec_x = worm_data['Center_x']
rec_y = worm_data['Center_y']
rec_time = worm_data['CenterTimes']

[center_length, center_time] = np.shape(rec_x)

In [ ]:
# Clean-up (removing bad frames)
if Worm == 5: # Bad frames at 905
    rec_x = np.delete(rec_x, 905, 1)
    rec_y = np.delete(rec_y, 905, 1)
    rec_time = np.delete(rec_time, 905, 0)

In [ ]:
# Plot centerline at all times
f = plt.figure(figsize=(20,10))
ax = plt.axes()

for i in range(0,center_time-1):
    ax.plot(rec_x[1:center_length-1,i], rec_y[1:center_length-1,i]) # plot without first and last point

ax.set_title("Worm Centerline through All Time")
ax.set_xlabel(r'x-position')
ax.set_ylabel(r'y-position')
plt.show()

## Animate centerlines

1 frame per 20 milliseconds (this is not in real time, it should be faster than real time)

**NOTE:** 
Playing animations inline and saving them require special software and python libraries
In order to work with animations at all, you need to have ffmpeg.
I recommend installing with brew: In terminal '*brew install ffmpeg*'
This should allow you to save your animations for replay outside of IPython (i.e. could
open in QuickTime or VLC)

For nifty inline animations, I use the Python library JSAnimation which you can install from:
**https://github.com/numerical-mooc/numerical-mooc/wiki/HOWTO:-Install-JSAnimation-for-IPython-Notebook**
Simply import the library as shown below. Note however that it you may only want to view a subset of the total frames (~200) - things start to get iffy beyond that.

General information about animation with matplotlib can be found at: ** http://matplotlib.sourceforge.net/api/animation_api.html**

In [ ]:
from JSAnimation import IPython_display

# First set up the figure, the axis, and the plot element we want to animate
f = plt.figure(figsize=(20,10))
ax = plt.axes(xlim=(0,1000), ylim=(0,1000))
line, = ax.plot([], [], lw=2)

# initialization function: plot the background of each frame
def init():
    line.set_data([], [])
    return line,

# animation function.  This is called sequentially
def animate(i):
    line.set_data(rec_x[1:center_length-1,i], rec_y[1:center_length-1,i])
    return line,

### For inline animation
animation.FuncAnimation(f, animate, init_func=init,
                        frames=100, interval=20, blit=True)

### For saving the animation as an mp4
# anim = animation.FuncAnimation(f, animate, init_func=init,
#                                frames=time, interval=20, blit=True)
# plt.close()
# anim.save('Worm' + str(Worm) + 'Centerline.mp4',
#           fps=30, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])

# Animation of centerlines with neural information

1 frame every 20 miliseconds (faster than real time)

* read in neural data
* line up neural data with times for centerline
* remove NaN values in both neural data and centerline data
* create plots of GCamp, RFP, centerline with vertical lines going across GCamp and RFP in sync with the centerline data. 

In [ ]:
# Clean neural data (remove NaN's) + clean recording data
G = worm_data['G']
R = worm_data['R']
Time = worm_data['Time']

badcols = np.array([x|y for (x,y) in zip(np.isnan(G).any(axis=0), np.isnan(R).any(axis=0))])
G = G[:,~badcols]
R = R[:,~badcols]
Time = Time[~badcols]

[neuron_length, neuron_time] = np.shape(G)
timePoints = np.arange(1,neuron_time)

f ,ax = plt.subplots(1,2, figsize=(20,5*2))
ax[0].imshow(G, aspect=10,cmap="Paired")
ax[1].imshow(R, aspect=10,cmap="Paired")
for axis, title in zip(ax, [r'GCAMP', r'RFP']):
    axis.set_title(title)
    axis.set_xlabel(r'Frame #')
    axis.set_ylabel(r'Neuron #')

## Match up times for neural and centerline data

Neural data is seperated by .166 sec on average

Centerline data is seperated by 0.016 sec on average

For each neural timestamp, find the index of centerline data that sits closest to it and record those indicies

In [ ]:
# Make sure centerline times encompass neuron times
if rec_time[0] > Time[0] or rec_time[-1] < Time[-1]:
    viable_indices = np.array([rec_time[0] < i and rec_time[-1] > i for i in Time])
    Time = Time[viable_indices]
    print('Neuron data was trimmed to only data with corresponding centerline info. \\
          Data now starts at ' + str(Time[0]) + 'sec and ends at ' + str(Time[-1]) + 'sec')
    
ind = np.zeros(len(Time), dtype=int)
i = 0
j = 0
for i in range(len(Time)):
    while rec_time[j] < Time[i]:
        j += 1
    if abs(rec_time[j-1] - Time[i]) < abs(rec_time[j] - Time[i]):
        ind[i] = j-1
    else:
        ind[i] = j
        
# Check
print('The average difference in time courses is ' + 
     '%.1f' % (np.average(abs(rec_time[ind] - Time))*1000) + 'msec and the max difference is ' +
     '%.1f' % (np.max(abs(rec_time[ind] - Time))*1000) + 'msec')

In [ ]:
from JSAnimation import IPython_display
%pylab inline

f_combo = plt.figure(figsize=(20,20))
ax1 = plt.subplot2grid((2,4), (0,0), colspan=2)
ax1.set_xlim([0, neuron_time])
ax1.set_ylim([0, neuron_length])
ax2 = plt.subplot2grid((2,4), (0,2), colspan=2)
ax2.set_xlim([0, neuron_time])
ax2.set_ylim([0, neuron_length])
ax3 = plt.subplot2grid((2,4), (1, 1), colspan=2)
ax3.set_xlim([0,1000])
ax3.set_ylim([0,1000])
plt.tight_layout()

# set up lines to plot
line_1_top, = ax1.plot([],[],lw=2, color='r')
line_2_top, = ax2.plot([],[],lw=2, color='r')
line_3, = ax3.plot([],[],lw=2)

# initalize
def init_combo():
    # background images
    ax1.imshow(G, aspect=10,cmap="Paired")
    ax2.imshow(R, aspect=10,cmap="Paired")
    # lines that change
    line_1_top.set_data([],[])
    line_2_top.set_data([],[])
    line_3.set_data([],[])
    return line_1_top, line_2_top, line_3,

def animate_combo(i):
    line_1_top.set_data(np.repeat(i,neuron_length), range(0,neuron_length))
    line_2_top.set_data(np.repeat(i,neuron_length), range(0,neuron_length))
    line_3.set_data(rec_x[1:center_length-1,ind[i]], rec_y[1:center_length-1,ind[i]])
    return line_1_top, line_2_top, line_3,

### For inline animation
# animation.FuncAnimation(f_combo, animate_combo, init_func=init_combo,
#                                frames=100, interval=20, blit=True)

### For saving the animation as an mp4
anim = animation.FuncAnimation(f_combo, animate_combo, init_func=init_combo,
                               frames=len(Time), interval=20, blit=True)
plt.close()
anim.save('worm' + str(Worm) + 'centerline_neural.mp4',
          fps=6, extra_args=['-vcodec', 'libx264', '-pix_fmt', 'yuv420p'])